# Evaluation pipeline for nearest neighbor search in remote sensing data

**Author:** [Lennart Seeger]<br>
**Date created:** 2021/04/24<br>
**Last modified:** 2023/03/24<br>

In [ ]:
from keras.applications.resnet import ResNet50
import keras
from tensorflow.keras import models, layers, Input
import os
import numpy as np
import umap
import sys
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import matplotlib.pyplot as plt
import copy
import random
from scipy.spatial.distance import cosine

sys.path.insert(1, '../src')
%load_ext autoreload
%autoreload 2

from supportive.reshape_images import reshape_images
from data.datasets import get_mlrsnet, get_denmark, get_ucmerced
from supportive.evaluate import evaluate_extractor, evaluate_extractor_classes
from supportive.visualize import visualize_brute_force, visualize_faiss
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

# 1. Parameter Initialisation, Model Loading, Dataset Loading, Feature Extraction

In [ ]:
# definition of parameters
image_size = 64
image_channels=3
dataset="denmark"
model="simclr" #feature_extraction, transfer_learning, simclr, nnclr, mae

# faiss parameters
num_neighbors=10
embedding_size = 2048
nlist = 20

In [ ]:
# SIMCLR
if model== "simclr":
    from models.simclr import get_augmenter
    path="../model/simclr/model"
    classification_augmentation = {"min_area": 0.75, "brightness": 0.3, "jitter": 0.1}
    model_loaded = keras.models.load_model(path)
    extractor_model = keras.Sequential(
            [
                layers.Input((image_size, image_size, 3)),
                get_augmenter(**classification_augmentation, image_size=image_size, image_channels=image_channels),
                model_loaded,
            ],
            name="extraction_model",
        )

In [ ]:
#NNCLR
if model == "nnclr":
    from models.nnclr import get_augmenter
    classification_augmentation = {
        "brightness": 0.2,
        "name": "classification_augmenter",
        "scale": (0.5, 1.0),
    }
    path="../model/nnclr/model"
    model_loaded = keras.models.load_model(path)
    extractor_model = keras.Sequential(
            [
                layers.Input((image_size, image_size, 3)),
                get_augmenter(**classification_augmentation, image_size=image_size),
                model_loaded,
            ],
            name="extraction_model",
        )

In [ ]:
if model == "feature_extraction":
    extractor_model = ResNet50(weights='imagenet', include_top=False,input_shape=(image_size,image_size,3),pooling="avg")        

In [ ]:
if model == "transfer_learning":
    transfer_model = keras.models.load_model('../model/transfer_learning/model')
    print("Feature extraction from the model")
    extractor_model = keras.Model(
       inputs=transfer_model.inputs,
       outputs=transfer_model.get_layer(name="my_intermediate_layer").output,
    )

In [ ]:
if model== "mae":
    from models.mae import get_test_augmentation_model, Patches, PatchEncoder
    path="../model/mae/model"
    model_loaded = keras.models.load_model(path)
    # mae parameters
    patch_size=4
    enc_projection_dim = 64
    mask_proportion=0.75
    patch_layer=Patches(patch_size)
    patch_encoder = PatchEncoder(patch_size, enc_projection_dim, mask_proportion)
    patch_encoder.downstream = True
    extractor_model = keras.Sequential(
    [
        layers.Input((image_size, image_size, 3)),
        get_test_augmentation_model(image_size),# try with and without
        patch_layer,
        patch_encoder,
        model_loaded,#encoder
        layers.BatchNormalization(),  # Refer to A.1 (Linear probing).
        layers.GlobalAveragePooling1D(),
    ],
    name="extraction_model",)

In [ ]:
if dataset=="mlrsnet":
    x_test, y_test = get_denmark(image_size=image_size)
    x_test = reshape_images(x_test)
    print(x_test.shape)
    print(y_test.shape)

In [ ]:
if dataset=="mlrsnet":
    y_test[y_test=="airplane"] = 0
    y_test[y_test=="airport"] = 1
    y_test[y_test=="bareland"] = 2
    y_test[y_test=="baseball_diamond"] = 3
    y_test[y_test=="basketball_court"] = 4
    y_test[y_test=="beach"] = 5
    y_test[y_test=="bridge"] = 6
    y_test[y_test=="chaparral"] = 7
    y_test[y_test=="cloud"] = 8
    y_test[y_test=="commercial_area"] = 9
    y_test[y_test=="dense_residential_area"] = 10
    y_test[y_test=="desert"] = 11
    y_test[y_test=="eroded_farmland"] = 12
    y_test[y_test=="farmland"] = 13
    y_test[y_test=="forest"] = 14
    y_test[y_test=="freeway"] = 15
    y_test[y_test=="golf_course"] = 16
    y_test[y_test=="ground_track_field"] = 17
    y_test[y_test=="harbor&port"] = 18
    y_test[y_test=="industrial_area"] = 19
    y_test[y_test=="intersection"] = 20
    y_test[y_test=="island"] = 21
    y_test[y_test=="lake"] = 22
    y_test[y_test=="meadow"] = 23
    y_test[y_test=="mobile_home_park"] = 24
    y_test[y_test=="mountain"] = 25
    y_test[y_test=="overpass"] = 26
    y_test[y_test=="park"] = 27
    y_test[y_test=="parking_lot"] = 28
    y_test[y_test=="parkway"] = 29
    y_test[y_test=="railway"] = 30
    y_test[y_test=="railway_station"] = 31
    y_test[y_test=="river"] = 32
    y_test[y_test=="roundabout"] = 33
    y_test[y_test=="shipping_yard"] = 34
    y_test[y_test=="snowberg"] = 35
    y_test[y_test=="sparse_residential_area"] = 36
    y_test[y_test=="stadium"] = 37
    y_test[y_test=="storage_tank"] = 38
    y_test[y_test=="swimming_pool"] = 39
    y_test[y_test=="swimmimg_pool"] = 39
    y_test[y_test=="tennis_court"] = 40
    y_test[y_test=="terrace"] = 41
    y_test[y_test=="transmission_tower"] = 42
    y_test[y_test=="vegetable_greenhouse"] = 43
    y_test[y_test=="wetland"] = 44
    y_test[y_test=="wind_turbine"] = 45
    
    classes_range = 46
    
    classesnames = ["airplane", "airport", "bareland", "baseball_diamond", "basketball_court", "beach", "bridge", "chaparral", "cloud","commercial_area", "dense_residential_area", "desert", "eroded_farmland", "farmland", "forest", "freeway", "golf_course", "ground_track_field", "harbor&port", "industrial_area", "intersection", "island", "lake", "meadow", "mobile_home_park", "mountain", "overpass", "park", "parking_lot", "parkway", "railway", "railway_station", "river", "roundabourt", "shipping_yard", "snowberg", "sparse_residential_area", "stadium", "storage_tank", "swimming_pool", "tennis_court", "terrace", "transmission_tower", "vegetable_greenhouse", "wetland", "wind_turbine"]

In [ ]:
if dataset=="denmark":
    train=np.load("../data/denmark.npz")
    x_test = reshape_images(train['images'])
    y_test = train['labels']
    print(x_test.shape)
    print(y_test.shape)

In [ ]:
if dataset=="denmark":
    y_test[y_test=="agricultural_brown"] = 0
    y_test[y_test=="agricultural_green"] = 1
    y_test[y_test=="big_buildings"] = 2
    y_test[y_test=="brown_buildings"] = 3
    y_test[y_test=="coast"] = 4
    y_test[y_test=="curve"] = 5
    y_test[y_test=="forest"] = 6
    y_test[y_test=="grey_buildings"] = 7
    y_test[y_test=="harbor"] = 8
    y_test[y_test=="highway"] = 9
    y_test[y_test=="lake"] = 10
    y_test[y_test=="parking_lot_crooked"] = 11
    y_test[y_test=="parking_lot_straight"] = 12
    y_test[y_test=="rail"] = 13
    y_test[y_test=="river"] = 14
    y_test[y_test=="road_buildings"] = 15
    y_test[y_test=="round_water"] = 16
    y_test[y_test=="silo_big"] = 17
    y_test[y_test=="silo_small"] = 18
    y_test[y_test=="single_road"] = 19
    y_test[y_test=="solar"] = 20
    y_test[y_test=="t_crossing"] = 21
    y_test[y_test=="tree_line"] = 22
    y_test[y_test=="water"] = 23
    y_test[y_test=="windmill"] = 24

    classes_range = 25

    classesnames = ["agricultural_brown", "agricultural_green", "big_buildings", "brown_buildings", "coast", "curve", "forest", "grey_buildings", "harbor", "highway", "lake", "parking_lot_crooked", "parking_lot_straight", "rail", "river", "road_buildings", "round_water", "silo_big", "silo_small", "single_road", "solar", "t_crossing", "tree_line", "water", "windmill"]

In [ ]:
# showing number of images per class
for i in range(classes_range):
    print(f"Examples of {i} ({classesnames[i]}) : {np.count_nonzero(y_test==i)}")

In [ ]:
x_test_original=x_test
if model == "feature_extraction":
    x_test_original=x_test
    x_test=preprocess_input(x_test)
y_test = y_test.astype("int32")
x_test_vectors=extractor_model.predict(x_test)
x_test_vectors.shape

In [ ]:
# calculating standard deviation for each class
for i in range(classes_range):
    sum=0
    x_test_class=x_test[y_test==i]
    print(classesnames[i])
    for element in x_test_class:
        sum += ((element[:,:,0].std()+element[:,:,1].std()+element[:,:,2].std())/3)
    print(sum/20)

In [ ]:
x_test=x_test[0:500]
y_test=y_test[0:500]

# 2. Calculating Neighbor Accuracy

In [ ]:
print("extractor_model: ", evaluate_extractor(extractor_model.predict, x_test, y_test, neighbors=10))

# 3. Calculating Neighbor Accuracy per Class

In [ ]:
evaluate_extractor_classes(extractor_model.predict, x_test, y_test, classesnames)

# 4. 2-D Visualisation of Feature Vectors

In [ ]:
reducer = umap.UMAP()
embedding = reducer.fit_transform(x_test_vectors)
embedding.shape

In [ ]:
colors = classesnames

fig, ax = plt.subplots(figsize=(10,5), dpi=300)
plt.scatter(embedding[:,0],embedding[:,1], c=y_test)
ax.tick_params(axis='both', which='major', labelsize=16)
cb = plt.colorbar()
loc = np.arange(0,max(y_test),max(y_test)/float(len(colors)))
cb.set_ticks(loc)
cb.set_ticklabels(colors)
fig.savefig(model+'_2d.png')

# 5. Neighbor Accuracy Mistakes per Class

In [ ]:
#returned classes
from scipy.spatial.distance import cosine
from supportive.evaluate import determine_predicted_classes
matrix=determine_predicted_classes(extractor_model, x_test, y_test, classesnames)
matrix.shape

In [ ]:
matrix = matrix.astype('int32')
matrix_root=np.sqrt(np.sqrt(matrix))
fig, ax = plt.subplots(figsize=(10, 10),dpi=100)
im = ax.imshow(matrix_root)#, cmap = 'magma' )

# Show all ticks and label them with the respective list entries
ax.set_xticks(np.arange(len(classesnames)), labels=classesnames)
ax.set_yticks(np.arange(len(classesnames)), labels=classesnames)

# Rotate the tick labels and set their alignment.
plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")

# Loop over data dimensions and create text annotations.
for i in range(len(classesnames)):
    for j in range(len(classesnames)):
        text = ax.text(j, i, matrix[i, j],
                       ha="center", va="center", color="w")

fig.tight_layout()
plt.show()
fig.savefig('classes-of-cutouts-predicted-as-neighbours-'+model+'.png')

# 6. Calculated Neighbors Visualized

## Brute Force Neighbours from testset

In [ ]:
x_test_vectors=extractor_model.predict(x_test)

In [ ]:
random_int = random.randrange(0,len(x_test))
min_list = []
for element in x_test_vectors:
    min_list.append(cosine(element,x_test_vectors[random_int]))
index_min = np.argmin(min_list)

indices_reduced = range(len(min_list))
a, indices_reduced = zip( *sorted( zip(min_list, indices_reduced)))

# plot original image as first
plt.figure(figsize=(20, 4),dpi=100)
ax = plt.subplot(2, 6, 1)
plt.imshow(x_test_original[random_int])
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)

# plot calculated nearest neighbors
for i in range(10):
    ax = plt.subplot(2, 6, i + 2)
    plt.imshow(x_test_original[indices_reduced[i]])
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

## Neighbors from Denmark

In [ ]:
x_final=np.load("../data/avg_std0.npy")

In [ ]:
x_final_vectors=np.zeros(embedding_size).reshape(1,embedding_size)
for i in range(0,len(x_final)//500):
    element=x_final[i*500:(i+1)*500]
    something=copy.deepcopy(element)
    if model=="feature_extraction":
        something= extractor_model(preprocess_input(something))
    something= extractor_model(preprocess_input(something))
    x_final_vectors=np.append(x_final_vectors, something, axis=0)
    print(i)

x_final_vectors=x_final_vectors[1:]
print(x_final_vectors.shape)

In [ ]:
random_int = random.randrange(0,len(x_final))
num_neighbors=100

### Brute Force Neighbors from Denmark

In [ ]:
visualize_brute_force(x_final_vectors, x_final, random_int)

### Approximate Neigbors from Denmark with FAISS

In [ ]:
sys.path.insert(1, '../src/faiss/build/faiss/python/')
import faiss
%load_ext autoreload
%autoreload 2

In [ ]:
x_final_vectors=x_final_vectors.astype('float32')

quantizer = faiss.IndexFlatL2(embedding_size)
index = faiss.IndexIVFFlat(quantizer, embedding_size, nlist, faiss.METRIC_L2)#METRIC_L2

assert not index.is_trained
index.train(x_final_vectors)
assert index.is_trained

index.add(x_final_vectors)

In [ ]:
D, I = index.search(x_final_vectors[random_int:random_int+1], num_neighbors)

In [ ]:
# Get indices of neighbors
min_list = []
for element in x_final_vectors:
    min_list.append(cosine(element,x_final_vectors[random_int]))

indices_reduced = range(len(min_list))
a, indices_reduced = zip( *sorted( zip(min_list, list(indices_reduced))))

In [ ]:
visualize_faiss(num_neighbors, x_final, random_int, index, x_final_vectors) 

# 7. Evaluate FAISS

In [ ]:
# Fraction of overlapping neighbors returned by FAISS and brute force
(num_neighbors-(len(set(list(I[0][0:num_neighbors])+list(indices_reduced[0:num_neighbors])))-num_neighbors))/num_neighbors

In [ ]:
# ratio how much worse the FAISS neighbors are

# brute force
counter_brute_force = 0
for element in indices_reduced[0:num_neighbors]:
    counter_brute_force += cosine(x_final_vectors[element],x_final_vectors[random_int])

# approximation
counter_approximations = 0
for element in I[0][0:num_neighbors]:
    counter_approximations += cosine(x_final_vectors[element], x_final_vectors[random_int])
    
print("ratio distances: "+ str(counter_approximations/counter_brute_force))